In [4]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".85"

import numpy as np

import jax
import jax.numpy as jnp
import spyx
import haiku as hk
import optax
from jax_tqdm import scan_tqdm

### NMNIST Dataloading

In [5]:

import tonic
from tonic import datasets, transforms
import torchvision as tv
from torch.utils.data import DataLoader, Subset
from collections import namedtuple

State = namedtuple("State", "obs labels")



In [6]:
sensor_size = tonic.datasets.NMNIST.sensor_size

# Denoise removes isolated, one-off events
# time_window
frame_transform = transforms.Compose([
                                      transforms.ToFrame(sensor_size=sensor_size, 
                                                         n_time_bins=64),
                                      lambda x: np.packbits(x, axis=0)
                                     ])

train_dataset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=frame_transform, train=True)
test_dataset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=frame_transform, train=False)

In [7]:
train_dl = iter(DataLoader(train_dataset, batch_size=len(train_dataset)//6,
                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=True))
        
x_train, y_train = next(train_dl)

In [8]:
#test_dl = iter(DataLoader(test_dataset, batch_size=len(test_dataset),
#                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=False))
#        
#x_test, y_test = next(test_dl)

In [9]:
x_train = jnp.array(x_train, dtype=jnp.uint8)
y_train = jnp.array(y_train, dtype=jnp.uint8)

#x_test = jnp.array(x_test, dtype=jnp.uint8)
#y_test = jnp.array(y_test, dtype=jnp.uint8)

In [10]:
def shuffle(dataset, shuffle_rng, batch_size):
    x, y = dataset

    cutoff = y.shape[0] % batch_size

    # perhaps calling rand.perm twice is slower than once and using indices?
    indices = jax.random.permutation(shuffle_rng, y.shape[0])[:-cutoff] # this is a bug if cutoff == 0
    obs, labels = x[indices], y[indices]

    obs = jnp.reshape(obs, (-1, batch_size) + obs.shape[1:])
    labels = jnp.reshape(labels, (-1, batch_size)) # should make batch size a global

    return State(obs=obs, labels=labels)

### Spyx NMNIST

In [11]:
def build_snn(batch_size, channel_multiplier):

    mult = channel_multiplier
    
    def nmnist_snn(x):

        x = hk.BatchApply(hk.Conv2D(12*mult, 5, padding="VALID", data_format="NCHW", with_bias=False))(x.astype(jnp.float32))

        core1 = spyx.nn.LIF((12*mult, 30, 30,), beta=0.5, activation=spyx.axn.Axon(spyx.axn.arctan()))
        x, V = hk.static_unroll(core1, x, core1.initial_state(x.shape[0]), time_major=False)

        fused1 = hk.Sequential([
            hk.MaxPool((2,2), (2,2), "VALID", channel_axis=-3),
            hk.Conv2D(32*mult, 5, padding="VALID", data_format="NCHW", with_bias=False)
        ])

        x = hk.BatchApply(fused1)(x)

        core2 = spyx.nn.LIF(( 32*mult, 11, 11,), beta=0.5, activation=spyx.axn.Axon(spyx.axn.arctan()))
        x, V = hk.static_unroll(core2, x, core2.initial_state(x.shape[0]), time_major=False)
        
        fused2 = hk.Sequential([
            hk.MaxPool((2,2), (2,2), "VALID", channel_axis=-3),
            hk.Flatten(),
            hk.Linear(10, with_bias=False)
        ])
        x = hk.BatchApply(fused2)(x)

        core3 = spyx.nn.LIF((10,), beta=0.5, activation=spyx.axn.Axon(spyx.axn.arctan()))
        spikes, V = hk.static_unroll(core3, x, core3.initial_state(x.shape[0]), time_major=False)
    
        return spikes, V
    
    key = jax.random.PRNGKey(0)
    # Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
    sample_x, sample_y = shuffle((x_train,y_train),key, batch_size)
    SNN = hk.without_apply_rng(hk.transform(nmnist_snn))
    params = SNN.init(rng=key, x=jnp.float32(sample_x[0]))
    
    return SNN, params




In [12]:
def benchmark(SNN, params, dataset, epochs, batch_size):
        
    opt = optax.adam(learning_rate=5e-4)
    
    # create and initialize the optimizer
    opt_state = opt.init(params)
    grad_params = params
        
    # define and compile our eval function that computes the loss for our SNN
    @jax.jit
    def net_eval(weights, events, targets):
        readout = SNN.apply(weights, events)
        traces, V_f = readout
        return spyx.fn.integral_crossentropy(traces, targets) # smoothing needs to be more explicit in docs...
        
    # Use JAX to create a function that calculates the loss and the gradient!
    surrogate_grad = jax.value_and_grad(net_eval) 
        
    rng = jax.random.PRNGKey(0)        
    
    # compile the meat of our training loop for speed
    @jax.jit
    def train_step(state, data):
        grad_params, opt_state = state
        events, targets = data 
        events = jnp.unpackbits(events, axis=1) # decompress temporal axis
        # compute loss and gradient                    # need better augment rng
        loss, grads = surrogate_grad(grad_params, events, targets)
        # generate updates based on the gradients and optimizer
        updates, opt_state = opt.update(grads, opt_state, grad_params)
        # return the updated parameters
        new_state = [optax.apply_updates(grad_params, updates), opt_state]
        return new_state, loss
    
    
    # Here's the start of our training loop!
    @scan_tqdm(epochs)
    def epoch(epoch_state, epoch_num):
        curr_params, curr_opt_state = epoch_state

        shuffle_rng = jax.random.fold_in(rng, epoch_num)
        train_data = shuffle(dataset, shuffle_rng, batch_size)
        
        # train epoch
        end_state, train_loss = jax.lax.scan(
            train_step,# func
            [curr_params, curr_opt_state],# init
            train_data,# xs
            train_data.obs.shape[0]# len
        )
                    
        return end_state, jnp.mean(train_loss)
    # end epoch
    
    # epoch loop
    start = time()
    final_state, metrics = jax.lax.scan(
        epoch,
        [grad_params, opt_state], # metric arrays
        jnp.arange(epochs), # 
        epochs # len of loop
    )
    
    final_params, loss = final_state
    final_params["LIF"]["beta"].block_until_ready()
    end = time() - start
    
                
    # return our final, optimized network.       
    return end

In [13]:
from time import time

def run_bench(trials, num_epochs, batch_size, mult):
    
    SNN, params = build_snn(batch_size, mult)

    # need to change how time is measured to match...
    times = []
    for t in range(trials+1):
        times.append(benchmark(SNN, params, (x_train,y_train), num_epochs, batch_size))
        print(times[t])
    
    print("Mean:", np.mean(times[1:]), "Std. Dev.:", np.std(times[1:]))
    return SNN, params

In [14]:
snn, p = run_bench(1, 20, 32, 1) # 160 seconds on laptop3060, 10k images.

  0%|          | 0/20 [00:00<?, ?it/s]

160.31551814079285


  0%|          | 0/20 [00:00<?, ?it/s]

154.2001552581787
Mean: 154.2001552581787 Std. Dev.: 0.0


In [ ]:
def viz(x):
    return snn.apply(p, x)

sample_x, sample_y = shuffle((x_train,y_train),jax.random.PRNGKey(0), 32)
print(hk.experimental.tabulate(viz)(sample_x[0]))

In [12]:
run_bench(1, 20, 128, 1)

(29952,)
0 :(29952,)


  0%|          | 0/20 [00:00<?, ?it/s]

149.29666376113892
1 :(29952,)


  0%|          | 0/20 [00:00<?, ?it/s]

143.64148688316345
Mean: 143.64148688316345 Std. Dev.: 0.0
